In [1]:
from math import trunc
from multiprocessing.sharedctypes import Value
import xml.etree.ElementTree as ET
import xml.dom.minidom
import os
import cv2
from tqdm import tqdm, trange
import numpy as np
import pandas as pd
import time
import warnings
warnings.filterwarnings("ignore")


from data_aug.data_aug import *
from data_aug.bbox_util import *

import matplotlib.pyplot as plt 
import pickle as pkl
%matplotlib inline



In [2]:
import os
if not os.path.exists('jittering_data'):
    os.mkdir('jittering_data')

In [3]:
origin_path = os.path.join('.', 'jittering_data', 'origin')
origin_Image_Path = os.path.join(origin_path, 'JPEGImages')
origin_Annotation_Path = os.path.join(origin_path, 'Annotations_with_ID')

output_path = os.path.join('.', 'jittering_data', 'output')
output_Image_Path = os.path.join(output_path, 'JPEGImages')
output_Annotation_Path = os.path.join(output_path, 'Annotations_with_ID')
if not os.path.exists(output_Image_Path):
    os.mkdir(output_Image_Path)
if not os.path.exists(output_Annotation_Path):
    os.mkdir(output_Annotation_Path)


In [4]:
target_list = [i.split('.')[0] for i in os.listdir(origin_Annotation_Path)[:]]

In [5]:
def process(target):
    origin_img_path = os.path.join(origin_Image_Path, target+'.jpg')
    origin_xml_path = os.path.join(origin_Annotation_Path, target+'.xml')
    origin_boxes = xml_to_boxes(origin_xml_path)
    origin_img = cv2.imread(origin_img_path)
    #origin_img = 255*np.ones(origin_img.shape, origin_img.dtype)
    #plt.imshow(origin_img[:,:,::-1])
    #plt.show()
    #origin_img2 = cv2.imread(origin_img_path)
    # Demo the data
    #print(origin_boxes)
    #plotted_img = draw_rect(origin_img, origin_boxes)
    #cv2.imwrite('nb.jpg', origin_img)
    #cv2.imwrite('nbt.jpg', origin_img2)
    #cv2.imshow('t',origin_img)
    #cv2.waitKey()
    #plt.show()
    # just copy
    #seq = Sequence([RandomHSV(0, 0, 100),RandomHorizontalFlip(), RandomScale(0.2), RandomRotate(20), RandomShear(0.2)])
    #seq = RandomHSV(50, 100, 100)
    #img_, bboxes_ = seq(origin_img.copy(),  origin_boxes.copy())
    #plt.imshow(img_[:,:,::-1])
    #plt.show()
    #return
    output_img_path =  os.path.join(output_Image_Path, target+str(0)+'.jpg')
    output_xml_path = os.path.join(output_Annotation_Path, target+str(0)+'.xml')
    cv2.imwrite(output_img_path, origin_img)
    for i in range(1,5):
        output_img_path =  os.path.join(output_Image_Path, target+str(i)+'.jpg')
        output_xml_path = os.path.join(output_Annotation_Path, target+str(i)+'.xml')
        
        seq = Sequence([RandomHSV(40, 50, 50),RandomHorizontalFlip(), RandomScale(0.2), RandomRotate(20), RandomShear(0.2)])
        #seq = RandomHSV(0, 0, 10)
        #ad = origin_img.copy()
        #print(ad.shape, ad.dtype)
        #cv2.imshow('title',origin_img)
        #cv2.waitKey()
        #cv2.imshow('title',ad)
        #cv2.waitKey()
        try:
            img_, bboxes_ = seq(origin_img.copy(),  origin_boxes.copy())
            cv2.imwrite(output_img_path, img_)
            tree = gen_tree(origin_xml_path, img_.shape,bboxes_)
            tree.write(output_xml_path)
        except:
            print("STOP")
        
        #plotted_img = draw_rect(img_, bboxes_)
        #plt.imshow(plotted_img[:,:,::-1])
        #plt.show()
    

    

In [6]:
def gen_tree(xml_path, img_shape, bboxes):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    root.find('size/width').text = str(img_shape[0])
    root.find('size/height').text = str(img_shape[1])
    boxes = [i for i in root.iter('object')]
    for box in boxes:
        root.remove(box)
    for box in bboxes:
        new_box = ET.Element('object')
        ET.SubElement(new_box, 'ID').text = str(int(box[4]))
        bndbox = ET.SubElement(new_box, 'bndbox')
        ET.SubElement(bndbox, 'xmin').text = str(int(box[0]))
        ET.SubElement(bndbox, 'ymin').text = str(int(box[1]))
        ET.SubElement(bndbox, 'xmax').text = str(int(box[2]))
        ET.SubElement(bndbox, 'ymax').text = str(int(box[3]))
        root.append(new_box)
    ET.indent(root)
    return tree

In [7]:

def xml_to_boxes(path):
    tree = ET.parse(path)
    root = tree.getroot()
    list_with_all_boxes = []
    for box in root.iter('object'):
        xmin = int(box.find("bndbox/xmin").text)
        ymin = int(box.find("bndbox/ymin").text)
        xmax = int(box.find("bndbox/xmax").text)
        ymax = int(box.find("bndbox/ymax").text)
        Id = int(box.find('ID').text)
        list_with_all_boxes.append([xmin, ymin, xmax, ymax, Id])
    return np.array(list_with_all_boxes, dtype='float64')


In [8]:
#process('000007')
for target in tqdm(target_list):
    process(target)

100%|██████████| 33546/33546 [48:06<00:00, 11.62it/s]  
